In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box
import rasterio
import os
from tqdm import tqdm
import numpy as np
import fiona
import pandas as pd 
from shapely.strtree import STRtree

## Part I: Creating our Graph

#### File-Paths
Understanding shapefiles: [Link](https://www.geowgs84.com/post/understanding-shapefiles-a-deep-dive-into-shp-dbf-shx-and-prj)

- `.shp`: points, lines and polygons; encoded in binary, ebgin with a fixed-length header and each feature then has its own header and content.
- `.shx`: spatial indexing - acts as an index for the `.shp` file - speeds up queries. Has fixed length records pointing to corresponding feature locations     
- `.prj`: projection file - contains the coordinate system and projection of our shapes
- `.dbf`: databased file - contains attribute data per shape - columns like name, population, elevation and the like.

Alternatives we may see: 
- GeoJSON 
- GPKG - Geopackage

*GeoPandas can be used for all vector files. Rasterio for all raster images*


In [ ]:
DATA_DIRECTORY = "data"
# Effis Fire Dataset - European Forest Fire Information System
# https://forest-fire.emergency.copernicus.eu/applications/data-and-services
FIRE_DATA_PATH = os.path.join(DATA_DIRECTORY, "fire_data/modis.ba.poly.shp")

# Several TIFFs with Wildfire Severity, once again from Effis
WILDFIRE_SEVERITY_DIR = os.path.join(DATA_DIRECTORY, "wildfire_severity")

# Natural Earth - public domain dataset of global geography
# Admin 0 – Countries dataset has country boundaries
# https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
COUNTRY_BOUNDARY_PATH = os.path.join(DATA_DIRECTORY, "country_boundaries/ne_110m_admin_0_countries.shp")

In [ ]:
coord_ref_sys = "EPSG:25829" 
# Coordinate reference system - defines how spatial data coordinates relate to the earth. Defines origin, units, orientation

### Filter out to our target area: Portugal

In [ ]:
# GeoPandas looks in the same directory for data files by default (.dbf, .shx, etc.)
fire_data = gpd.read_file(FIRE_DATA_PATH)
fire_data = fire_data.to_crs(coord_ref_sys) # reproject to our fixed CRS
fire_data = fire_data[fire_data["COUNTRY"] == "PT"]
# fire_data = fire_data.iloc[:100]
fire_data.head()
fire_data["FIREDATE"].nunique()

In [ ]:
fire_data.head()
# fire_data["FIREDATE"].nunique()

In [ ]:
world = gpd.read_file(COUNTRY_BOUNDARY_PATH)
portugal = world[world["NAME"] == "Portugal"]

# https://epsg.io/25829
# Reprojection of Portugal to our fixed CRS (specified above)
portugal_proj = portugal.to_crs(coord_ref_sys) # degrees -> met
mainland = portugal_proj.iloc[portugal_proj.geometry.area.argmax()]
mainland_polygon = mainland.geometry

# Extract bordering box around Portugal
minx, miny, maxx, maxy = mainland_polygon.bounds
print(minx, miny, maxx, maxy)

In [ ]:
grid_size = 10000 # meters - discretization size of what one node is in our graph
x_coords = np.arange(minx, maxx, grid_size)
y_coords = np.arange(miny, maxy, grid_size)

In [ ]:
# create grid
grid_squares = []
for x in x_coords:
    for y in y_coords:
        # this makes a full rectangle grid
        cell = box(x, y, x + grid_size, y + grid_size)
        # only add cells that are inside mainland portugal (leave out the sea)
        if mainland_polygon.contains(cell):
            grid_squares.append(cell)

grid = gpd.GeoDataFrame({"geometry": grid_squares}, crs=coord_ref_sys)
print(f"Num nodes = {len(grid_squares)}")
grid.head()
# each entry is lower-left, upper-left, upper-right, lower-right, lower-left points of each square

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))
grid.plot(ax=ax, edgecolor='black', facecolor='lightblue')
ax.ticklabel_format(style='sci', scilimits=(4,4))
ax.set_title("Portugal Grid - Shapefile")
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
plt.savefig("portugal_grid_plot.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
OUTPUT_DIRECTORY = 'output'
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)
grid.to_file(os.path.join(OUTPUT_DIRECTORY, "portugal_grid.shp"))
grid.to_file(os.path.join(OUTPUT_DIRECTORY,"portugal_grid.gpkg"), layer="grid", driver="GPKG")

In [ ]:
# add columns for coordinates of grid squares' centers
grid_gdf = grid.copy()
#rename geometry to geometry grid
# grid_gdf = grid_gdf.rename(columns={'geometry': 'geometry_grid'})
grid_gdf["centroid_grid"] = grid.geometry.centroid
grid_gdf["centroid_x_grid"] = grid.geometry.centroid.x
grid_gdf["centroid_y_grid"] = grid.geometry.centroid.y
grid_gdf.head()

In [ ]:
# Path to GDB of Water
gdb_path = os.path.join(DATA_DIRECTORY, "rivers", "euhydro_tajo_v013.gdb")

# List of layer indices we need
layer_indices = [0, 1, 6, 8, 9]

# Read and concatenate all water layers
water_gdfs = []
for idx in layer_indices:
    layer_name = fiona.listlayers(gdb_path)[idx]
    gdf_layer = gpd.read_file(gdb_path, layer=layer_name)
    water_gdfs.append(gdf_layer)

# Combine all layers into a single GeoDataFrame
water_gdf = gpd.GeoDataFrame(pd.concat(water_gdfs, ignore_index=True))
print("Combined water bodies:", water_gdf.shape)

grid_gdf = grid_gdf.set_geometry('centroid_grid')

# Ensure the CRS matches the water layer
grid_gdf = grid_gdf.to_crs(water_gdf.crs)

water_gdf = water_gdf[water_gdf.geometry.notnull()]
water_geoms = water_gdf.geometry.values
water_tree = STRtree(water_gdf.geometry.values)

def nearest_water(point):
    if point is None or point.is_empty:
        return None
    # nearest() now returns index
    nearest_idx = water_tree.nearest(point)
    nearest_geom = water_geoms[nearest_idx]  # get the actual geometry
    return point.distance(nearest_geom)

grid_gdf['dist_to_water'] = grid_gdf.geometry.apply(nearest_water)
grid_gdf.to_crs(coord_ref_sys, inplace=True)
grid_gdf.head()


In [ ]:
grid_gdf = grid_gdf.reset_index(drop=True)
grid_gdf["node_id"] = np.arange(len(grid_gdf))
grid_gdf.head()

#### Overlay with Fire Data

Assumption - if a centroid overlaps with a fire point - this is our fire node. We want to make this into a series, per date, for each fire.

In [ ]:
# fires_with_centroids["centroid_x"] = fire_data["GEOMETRY"]
fire_data["centroid_x_fire"] = fire_data['geometry'].apply(lambda x: x.centroid.x)
fire_data["centroid_y_fire"] = fire_data['geometry'].apply(lambda y: y.centroid.y)
fire_data.head()

In [ ]:
from datetime import datetime
import pandas as pd

fire_data["DAY"] = pd.to_datetime(fire_data["FIREDATE"], format='mixed').dt.date
# fire_data["FIREDATE"].nunique()
# fire_data["DAY"].nunique()
grouped_by_day = fire_data.groupby("DAY").size().reset_index(name='fire_count')
grouped_by_day.head()
grouped_by_day.shape


In [ ]:
 def safe_stat(value, func="mean"):
    """Safely compute a statistic from a pandas Series, scalar, or missing value."""
    if value is None or isinstance(value, (float, int, np.generic, type(pd.NA))):
        return value if isinstance(value, (float, int)) else None
    
    try:
        if func == "mean":
            return value.mean()
        elif func == "sum":
            return value.sum()
        elif func == "mode":
            mode = value.mode()
            return mode.iloc[0] if not mode.empty else None
    except Exception:
        return None


Clustering in terms of weather data to reduce number of API calls

### Aggregate Grid Based on Day

# Important: This is currently set to break after one step

In [ ]:
from meteostat import Point, Hourly
from datetime import timedelta
import pandas as pd
from tqdm import tqdm
import numpy as np


fire_data = fire_data.to_crs(grid_gdf.crs)
graphs = []
for day in tqdm(grouped_by_day["DAY"]):
    fire_data_day = fire_data[fire_data["DAY"] == day].copy()
    fire_data_day['geometry'] = fire_data_day.geometry.centroid

    fire_centroids = fire_data_day.copy()
    fire_centroids['geometry'] = fire_data_day.geometry.centroid
    
    # Perform spatial join: fire centroid within grid polygons - Faster than iterating over each centroid - https://www.youtube.com/watch?v=y85IKthrV-s
    # We join POINT with POLYGON

    grid_gdf = grid_gdf.set_geometry("geometry")
    fire_centroids = fire_centroids.set_geometry("geometry")
    joined = gpd.sjoin(grid_gdf, fire_centroids, how="left", predicate="covers")

    agg = (
        joined
        .assign(_hit = joined["index_right"].notna().astype(int))
        .groupby(level=0)  # group by grid cell index
        .agg(fire_count=("index_right", "count"),
             has_fire=("_hit", "max"))
    )

    # Merge back to geometry; fill missing (no matches) with zeros
    joined_one_row = grid_gdf.join(agg, how="left").fillna({"fire_count": 0, "has_fire": 0})
    joined_one_row["fire_count"] = joined_one_row["fire_count"].astype(int)
    joined_one_row["has_fire"]   = joined_one_row["has_fire"].astype(int)

    joined_one_row["node_id"] = grid_gdf["node_id"].values
    joined_one_row["DAY"] = pd.to_datetime(day).normalize()

    graphs.append(joined_one_row)
    # joined["geometry_centroid"] = joined.geometry.centroid
    # coords = [(geom.x, geom.y) for geom in joined.geometry_centroid]
    #
    # # Note: this method does not handle an edge case where multiple fires occured at the same place,
    # joined["fire_intensity"] = 0
    #
    # for filename in os.listdir(WILDFIRE_SEVERITY_DIR):
    #     if filename.endswith(".tiff"):
    #         filepath = os.path.join(WILDFIRE_SEVERITY_DIR, filename)
    #         with rasterio.open(filepath) as src:
    #
    #             # Ensure CRS match
    #             if joined.crs != src.crs:
    #                 joined = joined.to_crs(src.crs)
    #                 joined["geometry_centroid"] = joined.geometry.centroid
    #                 coords = [(geom.x, geom.y) for geom in joined.geometry_centroid]
    #
    #             values = np.array([val[0] for val in src.sample(coords)])
    #             joined["fire_intensity"] = np.maximum(joined["fire_intensity"], values)
    #
    # print(joined.columns)
    # results = []
    # i = 0
    # for point, date in tqdm(zip(joined["geometry_centroid"], joined["DAY"]), total=len(joined)):
    #     if point is None or pd.isna(date):
    #         continue
    #
    #     location = Point(point.y, point.x)
    #
    #     # UTC timestamps for Meteostat
    #     start = pd.Timestamp(date)
    #     end = start + timedelta(days=1)
    #
    #     # Fetch hourly data
    #     df = Hourly(location, start, end).fetch()
    #     if df.empty:
    #         i+=1
    #
    #     if not df.empty:
    #         # Aggregate all hours in that day
    #         results.append({
    #             "DAY": date,
    #             "lat": point.y,
    #             "lon": point.x,
    #             "temp_mean": safe_stat(df["temp"], "mean"),
    #             "rhum_mean": safe_stat(df["rhum"], "mean"),
    #             "wdir_mean": safe_stat(df["wdir"], "mean"),
    #             "wspd_mean": safe_stat(df["wspd"], "mean"),
    #             "pres_mean": safe_stat(df["pres"], "mean"),
    #         })
    #
    # weather_df = pd.DataFrame(results)
    # joined = pd.concat([joined, weather_df], axis=1)
    #
    # graphs.append(joined_one_row)


In [ ]:
joined_one_row

## Convert to .npy format with one feature (number of

#### Distance Matrix

In [ ]:
cent = grid_gdf.geometry.centroid
N = len(cent)
A = np.zeros(shape = (N, N))

grid_points = grid_gdf["geometry"].to_numpy()
P1s = np.array(list(map(lambda x: np.array([x.centroid.x, x.centroid.y]), grid_points[:] )))
P2s = np.array(list(map(lambda x: np.array([x.centroid.x, x.centroid.y]), grid_points[:] )))

A = np.sum((P1s[:,None,:] - P2s[None,:,:])**2, axis=2)**0.5

In [ ]:
np.save("data/distance_matrix.npy", A)

#### Timeseries Data

In [ ]:
# Construct Timeseries Data
# Concatenate all days into one long table (keep only what we need)
long = pd.concat(
    [
        g[["node_id", "fire_count", "DAY"]].copy()
          .assign(DAY=pd.to_datetime(g["DAY"]).dt.normalize())  # ensure consistent dtype
        for g in graphs
    ],
    axis=0,
    ignore_index=True
)

# If any duplicates slipped in per (node_id, DAY), sum them:
long = (long
        .groupby(["node_id", "DAY"], as_index=False)["fire_count"]
        .sum())

# Create a node index and sorted day index:
node_index = (grid_gdf[["node_id"]]
              .drop_duplicates()
              .sort_values("node_id")
              .set_index("node_id")
              .index)

day_index = (long["DAY"]
             .dropna()
             .sort_values()
             .unique())

#Pivot to wide matrix (N x T), fill missing vals with 0:
wide = (long
        .pivot_table(index="node_id", columns="DAY", values="fire_count", aggfunc="sum", fill_value=0)
        .reindex(index=node_index, columns=day_index, fill_value=0)
        .sort_index(axis=0)
        .sort_index(axis=1))

#Final (N,T) float32 array:
timeseries_data = wide.to_numpy(dtype=np.float32)  # shape (10041, 1284)

np.save("data/timeseries_data.npy", timeseries_data)
np.save("data/days.npy", day_index.astype("datetime64[ns]"))
# grid_gdf[["centroid_x_grid", "centroid_y_grid"]].describe()

In [ ]:
# from scipy.sparse import save_npz
# from sklearn.neighbors import radius_neighbors_graph
# Construct Distance Matrix



    
# print(A)
    
    
# XY = np.c_[cent.x.values, cent.y.values]
# N = XY.shape[0]

# # Estimate cell size from centroid grid
# ux = np.unique(np.round(XY[:,0], 6))
# dx = np.median(np.diff(np.sort(ux)))
# radius_m = 4 * dx


# A = radius_neighbors_graph(
#     XY, radius=radius_m, mode="connectivity", include_self=False
# ).tocsr().astype(np.uint8)

# A = ((A + A.T) > 0).astype(np.uint8)

# save_npz("data/adjacency_radius_4cells.npz", A)
# print("adj nnz:", A.nnz, "avg deg:", A.nnz / N)



## 2nd try with clustering for weather data
To reduce the number of API calls

In [ ]:
from sklearn.cluster import KMeans

grid_gdf.to_crs(coord_ref_sys)
coords = np.array([(geom.y, geom.x) for geom in grid_gdf.centroid_grid])

# clustering based on geographic closeness
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init = 10)
grid_gdf["cluster_id"] = kmeans.fit_predict(coords)

fig, ax = plt.subplots(figsize=(10,10))
grid_gdf.plot(column="cluster_id", categorical=True, ax=ax, cmap="tab20")
grid_gdf.geometry.centroid.plot(ax=ax, color="black", markersize=5, alpha=0.6)
ax.set_title("KMeans Clusters of Grid Squares")
ax.set_axis_off()
plt.savefig("kmeans_weather.png", dpi=300, bbox_inches="tight")
plt.show()

grid_gdf.head()

In [ ]:
from meteostat import Point, Hourly
from datetime import timedelta

start = pd.Timestamp(fire_data["DAY"].min())
end = pd.Timestamp(fire_data["DAY"].max() + timedelta(days=1))

cluster_weather = []

for cluster_id, group in grid_gdf.groupby("cluster_id"):
    # Fetch weather data for each cluster (instead of every grid square)
    lat = group.geometry.centroid.y.mean()
    lon = group.geometry.centroid.x.mean()

    # fetch weather data for the full date range
    df_weather = Hourly(Point(lat, lon), start, end).fetch()

    if not df_weather.empty:
        daily = df_weather.resample("D").mean().reset_index() # daily mean
        daily["DAY"] = daily["time"].dt.date
        daily["cluster_id"] = cluster_id
        # temperature, relative humidity, wind direction, wind speed, air pressure
        cluster_weather.append(daily[["cluster_id", "DAY", "temp", "rhum", "wdir", "wspd", "pres"]])

weather_df = pd.concat(cluster_weather, ignore_index=True)
weather_df.to_csv("cluster_weather.csv", index=False)

weather_df.head()

### Original Fire Attributes
- **`id`** → Unique identifier for each fire record.  
- **`FIREDATE`** → Date and time when the fire occurred.  
- **`LASTUPDATE`** → Timestamp of the last update for the fire record.  
- **`COUNTRY`** → Country where the fire occurred.  
- **`PROVINCE`** → Province of the fire location.  
- **`COMMUNE`** → Commune (local administrative area) of the fire.  
- **`AREA_HA`** → Burned area in hectares.  

### Land Cover / Fire Area Composition
- **`BROADLEA`** → Proportion of broadleaf vegetation in the fire area.  
- **`CONIFER`** → Proportion of coniferous vegetation.  
- **`MIXED`** → Proportion of mixed forest.  
- **`SCLEROPH`** → Proportion of sclerophyllous vegetation (e.g., Mediterranean shrubs).  
- **`TRANSIT`** → Proportion of transitional land cover.  
- **`OTHERNATLC`** → Proportion of other natural land cover.  
- **`AGRIAREAS`** → Proportion of agricultural areas.  
- **`ARTIFSURF`** → Proportion of artificial surfaces (urban/industrial).  
- **`OTHERLC`** → Proportion of other land cover types.  
- **`PERCNA2K`** → Percentage of area under special designation (e.g., Natura 2000).  
- **`CLASS`** → Fire classification or severity category.  

### Fire Geometry / Location
- **`geometry`** → Original fire geometry (polygon or point).  
- **`centroid_x_fire`** → X coordinate of fire centroid.  
- **`centroid_y_fire`** → Y coordinate of fire centroid.  

### Spatial Join with Grid Cells
- **`index_right`** → Index of the grid cell containing the fire centroid (NaN if none).  
- **`centroid_grid`** → Geometry of the grid cell centroid.  
- **`centroid_x_grid`** → X coordinate of the grid cell centroid.  
- **`centroid_y_grid`** → Y coordinate of the grid cell centroid.  
- **`has_fire`** → Boolean indicating if the fire is inside a grid cell.  

### Time-Based Helper
- **`DAY`** → Date only (no time), extracted from `FIREDATE`, used for grouping fires by day.


In [ ]:
# A summary and sample of the graphs list - + persistence
import pickle
with open('output/graphs.pkl', 'wb') as f:
    pickle.dump(graphs, f)
    
print("Number of time steps(graphs)", len(graphs))
print("Example graph for one day:")
graphs[0]

## Attempt to Convert to .npy Format for the Models

In [ ]:
date_col = "FIREDATE"   # adjust to your real column name
fire_data[date_col] = pd.to_datetime(fire_data[date_col]).dt.normalize()
full_days = pd.date_range(
    fire_data[date_col].min(),
    fire_data[date_col].max(),
    freq="D"
)